In [1]:
# imports
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [2]:
# functions
def fetch_data(year):
    """Fetch offensive and defensive data for given year"""
    url_offensive = f"https://stats.inpredictable.com/nba/ssnTeamPoss.php?season={year}"
    url_defensive = f"https://stats.inpredictable.com/nba/ssnTeamPoss.php?season={year}&view=def"
    
    df_offensive = pd.read_html(url_offensive)[0]
    df_defensive = pd.read_html(url_defensive)[0]
    
    # Format column names
    df_offensive.columns = [' '.join(col).strip() for col in df_offensive.columns.values]
    df_defensive.columns = [' '.join(col).strip() for col in df_defensive.columns.values]
    
    return df_offensive, df_defensive

def clean_percentage_columns(df):
    """Convert percentage columns to float values"""
    df['After Def Rebound %'] = df['After Def Rebound %'].str.rstrip('%').astype('float') / 100
    df['After Made Shot* %'] = df['After Made Shot* %'].str.rstrip('%').astype('float') / 100
    return df

def calculate_non_fastbreak_rating(df, rating_column_name):
    """Calculate non-fastbreak rating"""
    df[rating_column_name] = df.apply(
        lambda row: (
            (row['After Def Rebound points'] * row['After Def Rebound %'] +
            row['After Made Shot* points'] * row['After Made Shot* %']) /
            (row['After Made Shot* %'] + row['After Def Rebound %'])
        ) if pd.notnull(row['After Def Rebound points']) and pd.notnull(row['After Made Shot* points']) else None, 
        axis=1
    )
    return df

def calculate_half_court_difference(df, col_name, total_points_col):
    """Calculate difference between half court and average possessions"""
    df[col_name] = df.apply(
        lambda row: (
            row[total_points_col] - row['After Made Shot* points']
        ),
        axis=1
    )
    return df

def process_year_data(year):
    """Process data for a specific year"""
    # Fetch data
    df_offensive, df_defensive = fetch_data(year)
    
    # Clean percentage columns
    df_offensive = clean_percentage_columns(df_offensive)
    df_defensive = clean_percentage_columns(df_defensive)
    
    # Calculate non-fastbreak ratings
    df_offensive = calculate_non_fastbreak_rating(df_offensive, 'non_fb_o_rating')
    df_defensive = calculate_non_fastbreak_rating(df_defensive, 'non_fb_d_rating')
    
    # Calculate half-court differences
    df_offensive = calculate_half_court_difference(df_offensive, 'o_half_court_dff', 'Total points&nbsp ▼')
    df_defensive = calculate_half_court_difference(df_defensive, 'd_half_court_dff', 'Total points')
    
    # Merge offensive and defensive data
    combined = pd.merge(df_offensive, df_defensive, on='Unnamed: 1_level_0 Team', how='left')
    combined['year'] = year
    
    # Create final dataframe for the year
    new_df = pd.DataFrame(combined.loc[combined.index[:-2], [
        'Unnamed: 1_level_0 Team', 'year', 'non_fb_o_rating', 'non_fb_d_rating', 
        'o_half_court_dff', 'd_half_court_dff'
    ]])
    new_df.rename(columns={'Unnamed: 1_level_0 Team': 'team'}, inplace=True)
    
    return new_df

In [10]:
# Process data for each year
for year in tqdm(range(2013, datetime.now().year)):
    # print(f'processing year: {year}')
    year_data = process_year_data(year)
    year_data.to_csv(f'data/team_adj/team_adj_20{year%2000}_{year%2000+1}.csv')

  0%|          | 0/12 [00:00<?, ?it/s]

processing year: 2013


  8%|▊         | 1/12 [00:06<01:16,  6.94s/it]

processing year: 2014


 17%|█▋        | 2/12 [00:12<01:00,  6.02s/it]

...

 83%|████████▎ | 10/12 [00:35<00:05,  2.93s/it]

processing year: 2023


 92%|█████████▏| 11/12 [00:37<00:02,  2.81s/it]

processing year: 2024


100%|██████████| 12/12 [00:40<00:00,  3.41s/it]
